# Disassembly

For disassembling Python byte code.

* See https://docs.python.org/3/library/dis.html
* See https://en.wikipedia.org/wiki/Stack-oriented_programming

Python is a stack-orientated language. There are 3 stacks: call, data (python code/function evaluation) and block (control structures).

Disassemble an object - module, class, method, function etc...

In [2]:
import dis

def my_function(x, y):
    _z = x + y
    print(_z)
    try:
        f = open('my_file.txt', 'r')
    except FileNotFoundError:
        pass
    return _z

dis.dis(my_function)

  4           0 LOAD_FAST                0 (x)
              2 LOAD_FAST                1 (y)
              4 BINARY_ADD
              6 STORE_FAST               2 (_z)

  5           8 LOAD_GLOBAL              0 (print)
             10 LOAD_FAST                2 (_z)
             12 CALL_FUNCTION            1
             14 POP_TOP

  6          16 SETUP_FINALLY            8 (to 34)

  7          18 LOAD_GLOBAL              1 (open)
             20 LOAD_CONST               1 ('my_file.txt')
             22 LOAD_CONST               2 ('r')
             24 CALL_FUNCTION            2
             26 STORE_FAST               3 (f)
             28 POP_BLOCK

 10          30 LOAD_FAST                2 (_z)
             32 RETURN_VALUE

  8     >>   34 DUP_TOP
             36 LOAD_GLOBAL              2 (FileNotFoundError)
             38 JUMP_IF_NOT_EXC_MATCH    26 (to 52)
             40 POP_TOP
             42 POP_TOP
             44 POP_TOP

  9          46 POP_EXCEPT

 10          48 LO

Show code information

In [3]:
print(dis.code_info(my_function))

Name:              my_function
Filename:          C:\Users\Mike\AppData\Local\Temp\ipykernel_9896\2776365920.py
Argument count:    2
Positional-only arguments: 0
Kw-only arguments: 0
Number of locals:  4
Stack size:        8
Flags:             OPTIMIZED, NEWLOCALS, NOFREE
Constants:
   0: None
   1: 'my_file.txt'
   2: 'r'
Names:
   0: print
   1: open
   2: FileNotFoundError
Variable names:
   0: x
   1: y
   2: _z
   3: f


We can also access code information through the `__code__` attribute of a function

In [4]:
print(my_function.__code__)
print("literals =", my_function.__code__.co_consts)             # any literals in the code
print("locals =", my_function.__code__.co_varnames)             # local variables
print("non-locals =", my_function.__code__.co_names)            # non-local names referenced

<code object my_function at 0x000002560DE2FE10, file "C:\Users\Mike\AppData\Local\Temp\ipykernel_9896\2776365920.py", line 3>
literals = (None, 'my_file.txt', 'r')
locals = ('x', 'y', '_z', 'f')
non-locals = ('print', 'open', 'FileNotFoundError')


Get instructions using an iterator

In [5]:
my_instructions = dis.get_instructions(my_function)
for instruction in my_instructions:
    print(f"opcode: {instruction.opcode} opname: {instruction.opname} arg: {instruction.arg}")

opcode: 124 opname: LOAD_FAST arg: 0
opcode: 124 opname: LOAD_FAST arg: 1
opcode: 23 opname: BINARY_ADD arg: None
opcode: 125 opname: STORE_FAST arg: 2
opcode: 116 opname: LOAD_GLOBAL arg: 0
opcode: 124 opname: LOAD_FAST arg: 2
opcode: 131 opname: CALL_FUNCTION arg: 1
opcode: 1 opname: POP_TOP arg: None
opcode: 122 opname: SETUP_FINALLY arg: 8
opcode: 116 opname: LOAD_GLOBAL arg: 1
opcode: 100 opname: LOAD_CONST arg: 1
opcode: 100 opname: LOAD_CONST arg: 2
opcode: 131 opname: CALL_FUNCTION arg: 2
opcode: 125 opname: STORE_FAST arg: 3
opcode: 87 opname: POP_BLOCK arg: None
opcode: 124 opname: LOAD_FAST arg: 2
opcode: 83 opname: RETURN_VALUE arg: None
opcode: 4 opname: DUP_TOP arg: None
opcode: 116 opname: LOAD_GLOBAL arg: 2
opcode: 121 opname: JUMP_IF_NOT_EXC_MATCH arg: 26
opcode: 1 opname: POP_TOP arg: None
opcode: 1 opname: POP_TOP arg: None
opcode: 1 opname: POP_TOP arg: None
opcode: 89 opname: POP_EXCEPT arg: None
opcode: 124 opname: LOAD_FAST arg: 2
opcode: 83 opname: RETURN_VALUE 

Comparing `[]` and `list()` and you can see why `[]` is faster 

In [10]:
dis.dis("[]")
dis.dis("list()")

  1           0 BUILD_LIST               0
              2 RETURN_VALUE
  1           0 LOAD_NAME                0 (list)
              2 CALL_FUNCTION            0
              4 RETURN_VALUE


Print `traceback` object

In [11]:
import traceback
import sys

my_list = [1, 2, 3]
try:
    print(my_list.womble)
except AttributeError as e:
    exc_type, exc_value, exc_traceback = sys.exc_info()
    traceback.print_tb(exc_traceback, limit=1, file=sys.stdout)
    print("type(e.__traceback__) =", str(type(e.__traceback__))[1:-1])
    traceback.print_tb(e.__traceback__, limit=1, file=sys.stdout)

  File "C:\Users\Mike\AppData\Local\Temp\ipykernel_9896\2710167046.py", line 6, in <module>
    print(my_list.womble)
type(e.__traceback__) = class 'traceback'
  File "C:\Users\Mike\AppData\Local\Temp\ipykernel_9896\2710167046.py", line 6, in <module>
    print(my_list.womble)


Print stack

In [8]:
def my_function():
    traceback.print_stack()

my_function()

  File "c:\python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\Mike\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\Mike\AppData\Roaming\Python\Python310\site-packages\traitlets\config\application.py", line 982, in launch_instance
    app.start()
  File "C:\Users\Mike\AppData\Roaming\Python\Python310\site-packages\ipykernel\kernelapp.py", line 712, in start
    self.io_loop.start()
  File "C:\Users\Mike\AppData\Roaming\Python\Python310\site-packages\tornado\platform\asyncio.py", line 215, in start
    self.asyncio_loop.run_forever()
  File "c:\python310\lib\asyncio\base_events.py", line 600, in run_forever
    self._run_once()
  File "c:\python310\lib\asyncio\base_events.py", line 1896, in _run_once
    handle._run()
  File "c:\python310\lib\a

Disassemble the last traceback in `my_function()`

In [9]:
def my_function():
    _my_list = [1, 2, 3]
    try:
        print(_my_list.womble)
    except AttributeError as e:
        dis.distb(e.__traceback__)

my_function()

  2           0 BUILD_LIST               0
              2 LOAD_CONST               1 ((1, 2, 3))
              4 LIST_EXTEND              1
              6 STORE_FAST               0 (_my_list)

  3           8 SETUP_FINALLY            8 (to 26)

  4          10 LOAD_GLOBAL              0 (print)
             12 LOAD_FAST                0 (_my_list)
    -->      14 LOAD_ATTR                1 (womble)
             16 CALL_FUNCTION            1
             18 POP_TOP
             20 POP_BLOCK
             22 LOAD_CONST               0 (None)
             24 RETURN_VALUE

  5     >>   26 DUP_TOP
             28 LOAD_GLOBAL              2 (AttributeError)
             30 JUMP_IF_NOT_EXC_MATCH    37 (to 74)
             32 POP_TOP
             34 STORE_FAST               1 (e)
             36 POP_TOP
             38 SETUP_FINALLY           13 (to 66)

  6          40 LOAD_GLOBAL              3 (dis)
             42 LOAD_METHOD              4 (distb)
             44 LOAD_FAST              

See https://github.com/python/cpython/blob/master/Include/opcode.h for op-code header file